In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd

import mesa
import mesa_geo as mg
import mesa_geo.visualization as mgv

from pathlib import Path

from model import GeoSIR
from utils import InfecStatus

# from model import GeoSIR
# from agents import PersonAgent, TractAgent
# from utils import TimeBlock, Decision, InfecStatus

# %load_ext autoreload
# %autoreload 2

In [2]:
data_path = Path().resolve() / 'data'
images_path = Path().resolve() / 'images'
images_path.mkdir(exist_ok=True)
flow_path = data_path / 'flow'

In [3]:
tracts_df = gpd.read_file(data_path / 'tracts.shp').to_crs(epsg=3857)
agents_tract_df = pd.read_csv(data_path / 'agents_tract.csv')
prob_stay_at_home_data = (
  pd.read_csv(
    data_path / 'agents_home.csv',
    parse_dates=['date'],
    date_format='%Y-%m-%d'
  )
  .assign(date=lambda x: x['date'].dt.date)
  .set_index(['date', 'tract'])
  .squeeze()
)
percentage_time_at_home_data = (
  pd.read_csv(
    data_path / 'agents_percentage_home.csv',
    parse_dates=['date'],
    date_format='%Y-%m-%d'
  )
  .assign(
    date=lambda x: x['date'].dt.date,
    percentage_time_home=lambda x: x['percentage_time_home'] / 100
  )
  .set_index(['date', 'tract'])
  .squeeze()
)

In [4]:
# Model
infection_params = {
  'beta': 0.5,
  'gamma': 1 / 7
}
initial_condition = {
  InfecStatus.S: 0.99,
  InfecStatus.I: 0.01,
  InfecStatus.R: 0.00
}
exposure_distance = 100
avg_trips = 2.6
model = GeoSIR(
  infection_params=infection_params,
  initial_condition=initial_condition,
  exposure_distance=exposure_distance,
  avg_trips=avg_trips,
  tracts_df=tracts_df,
  agents_tract_df=agents_tract_df,
  prob_stay_at_home_data=prob_stay_at_home_data,
  percentage_time_at_home_data=percentage_time_at_home_data,
  flow_path=flow_path,
  seed=42
)

TypeError: GeoSIR.__init__() got an unexpected keyword argument 'infection_params'

In [13]:
from mesa.batchrunner import _make_model_kwargs
parameters = {
  "infection_params": [infection_params],
  # "initial_condition": initial_condition,
  "exposure_distance": range(1, 2),
  "avg_trips": avg_trips,
  "tracts_df": [tracts_df],
  # "agents_tract_df": agents_tract_df,
  # "prob_stay_at_home_data": prob_stay_at_home_data,
  # "percentage_time_at_home_data": percentage_time_at_home_data,
  # "flow_path": flow_path,
}
kwargs_list = _make_model_kwargs(parameters)
kwargs_list

[{'infection_params': {'beta': 0.5, 'gamma': 0.14285714285714285},
  'exposure_distance': 1,
  'avg_trips': 2.6,
  'tracts_df':            tract                                           geometry
  0    53033025701  POLYGON ((-13603957.518 6019078.619, -13603956...
  1    53033025702  POLYGON ((-13601668.12 6018543.439, -13601661....
  2    53033025804  POLYGON ((-13601677.799 6016990.754, -13601674...
  3    53033026001  POLYGON ((-13608766.87 6023964.184, -13608752....
  4    53033026100  POLYGON ((-13611098.788 6023344.991, -13611067...
  ..           ...                                                ...
  388  53033002400  POLYGON ((-13613930.964 6055474.802, -13613821...
  389  53033002600  POLYGON ((-13616331.234 6055196.31, -13616184....
  390  53033002900  POLYGON ((-13621725.217 6054076.493, -13621718...
  391  53033009400  POLYGON ((-13616301.243 6039633.309, -13616260...
  392  53033009500  POLYGON ((-13614703.916 6038406.265, -13614673...
  
  [393 rows x 2 columns]}]

In [8]:
kwargs_list

[{'infection_params': 'beta', 'exposure_distance': 1, 'avg_trips': 2.6},
 {'infection_params': 'gamma', 'exposure_distance': 1, 'avg_trips': 2.6}]

In [ ]:
for _ in range(30):
  model.step()

In [ ]:
model.datacollector.get_model_vars_dataframe()

In [ ]:
parameters = {
  "infection_params": infection_params,
  "initial_condition": initial_condition,
  "exposure_distance": range(1, 4),
  "avg_trips": avg_trips,
  "tracts_df": tracts_df,
  "agents_tract_df": agents_tract_df,
  "prob_stay_at_home_data": prob_stay_at_home_data,
  "percentage_time_at_home_data": percentage_time_at_home_data,
  "flow_path": flow_path,
}

results = mesa.batch_run(
    GeoSIR,
    parameters,
    iterations=5,
    max_steps=100,
    data_collection_period=1,
    number_processes=3
)

In [ ]:
infection_params=infection_params,
initial_condition=initial_condition,
exposure_distance=exposure_distance,
avg_trips=avg_trips,
tracts_df=tracts_df,
agents_tract_df=agents_tract_df,
prob_stay_at_home_data=prob_stay_at_home_data,
percentage_time_at_home_data=percentage_time_at_home_data,
flow_path=flow_path,
seed=42

In [ ]:
model._agents_by_type[PersonAgent]

In [ ]:
dir(model)

In [ ]:
Decision['STAY_HOME']

In [ ]:
model._agents_by_type[PersonAgent][0].decision

In [ ]:
model.agent_types

In [ ]:
model.datacollector.get_model_vars_dataframe()


In [ ]:
model.counts

In [ ]:
for klass, l in model._agents_by_type.items():
  print(klass, len(l))

In [ ]:
model._agents_by_type[PersonAgent]#.groupby('status').count()

In [ ]:
model.flow_dates[-1:0]

In [ ]:
model.time_block

In [ ]:
model.time_block is TimeBlock.EVENING

In [ ]:
type(model._agents_by_type[PersonAgent][0].geometry)

In [ ]:
model._agents_by_type[PersonAgent][0].pos

In [ ]:
model._agents_by_type[TractAgent][0].prob_stay_at_home

In [ ]:
model.today

In [ ]:
# (
#   model.tracts_df.merge(
#     model.agents_tract_df,
#     how='left',
#     on='tract',
#     validate='1:1'
#   )
#   .rename(columns={'tract': 'code', 'n_agents': 'population'})
# )

In [ ]:
set(InfecStatus)

In [ ]:
InfecStatus.S in InfecStatus

In [ ]:
InfecStatus

In [ ]:
InfecStatus.S in set(InfecStatus)

In [ ]:
type(InfecStatus)

In [ ]:
members = list(TimeBlock)

In [ ]:
members.index(TimeBlock.MORNING)

In [ ]:
counts = {
      "S": 0,
      "I": 0,
      "R": 0,
    }

In [ ]:
InfecStatus.S in counts.keys()

In [ ]:
counts = {status: 0 for status in InfecStatus}
counts

In [ ]:
counts.keys()

In [ ]:
list(InfecStatus)

In [ ]:
InfecStatus.__members__.values()

In [ ]:
counts[InfecStatus.S]

In [ ]:
print(InfecStatus.S in counts)

In [ ]:
from enum import Flag, auto

class InfecStatus(Flag):
    S = auto()
    I = auto()
    R = auto()

counts = {status: 0 for status in InfecStatus}
print(counts[InfecStatus.S])  # Should work

In [ ]:
model.prob_stay_at_home_data

In [ ]:
model.today

In [ ]:
model.prob_stay_at_home_data.index.dtype

In [ ]:
my_date = percentage_time_at_home_data.index.get_level_values(0)[0]

In [ ]:
model.prob_stay_at_home_data.loc[model.current_flow_date, 53033032326]

In [ ]:
model.current_flow_date

In [ ]:
model._agents_by_type[PersonAgent][0].tract_id

In [ ]:
model.agents[1].prob_stay_at_home

In [ ]:
np.random.rand()

In [ ]:
tract_id = 1
tract = model.agents[tract_id]
tract.prob_flow

In [ ]:
model.random.choices(list(tract.prob_flow.keys()), weights=list(tract.prob_flow.values()))[0]

In [ ]:
InfecStatus._member_names_

In [ ]:
model.agents[1].geometry

In [ ]:
gpd.GeoSeries(model.agents[1].geometry).sample_points(size=(1, 2)).iat[0]